In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
from phik.report import plot_correlation_matrix

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, make_scorer, precision_score, recall_score

import shap

Matplotlib is building the font cache; this may take a moment.


In [2]:
!pip install phik

In [3]:
!pip install shap

In [4]:
pip install --upgrade scikit-learn matplotlib numpy seaborn shap

  Using cached numpy-2.0.2-cp39-cp39-macosx_10_9_x86_64.whl (21.2 MB)
Note: you may need to restart the kernel to use updated packages.


### Введение
В проекте предстоит проанализировать характеристики пользователей и их активность, чтобы предсказать вероятность её снижения, а также разбить пользователей на группы, для привлечения каждой из которых будет придумана своя стратегия. Для этого предстоит:

1. Выполнить предобработку
2. Добавить новые признаки
3. Провести анализ зависимостей признаков
4. Выбрать модель, наиболее подходящую для решения задачи и гиперпараметры к ней
5. Проанализировать вклад каждого признака на целевой
6. Разделить пользователей на сегменты и продумать стратегию для каждого

## Загрузка данных


In [5]:
market_file = pd.read_csv('/datasets/market_file.csv')
market_money = pd.read_csv('/datasets/market_money.csv')
market_time = pd.read_csv('/datasets/market_time.csv')
money =  pd.read_csv('/datasets/money.csv', sep=';', decimal=',')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/market_file.csv'

In [ ]:
market_file.head()

In [ ]:
market_money.head()

In [ ]:
market_time.head()

In [ ]:
money.head()

## Предобработка данных

In [ ]:
df_list = [market_file, market_money, market_time, money]

In [ ]:
for i in df_list:
    print(i.info(), '\n')

Типы данных в порядке, пропусков нет. Изменим названия признаков

In [ ]:
def rename_col(df):
        df.columns = df.columns.str.lower()
        df.columns = df.columns.str.replace(' ', '_')

In [ ]:
rename_col(market_file)
rename_col(market_money)
rename_col(market_time)
rename_col(money)

Проверим дубликаты

In [ ]:
for i in df_list:
    print(i.duplicated().sum())

In [ ]:
for i in df_list:
    for j in i.select_dtypes('object'):
        print(j, i[j].unique(), '\n')
    

Есть ошибка в признаке 'тип_сервиса' и 'период'

In [ ]:
market_file.loc[market_file['тип_сервиса'] == 'стандартт', 'тип_сервиса'] = 'стандарт'

In [ ]:
market_time.loc[market_time['период'] == 'предыдцщий_месяц', 'период'] = 'предыдущий_месяц'

## Исследовательский анализ данных

In [ ]:
for i in df_list:
    display(i.describe())

Замечен выброс в признаке 'выручка'

#### Построим графики 

In [ ]:
def num_hist(df, lst):
    for i in lst:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
        
        df[i].plot(kind='hist', bins=15, ax=axes[0])
        axes[0].set_title(f'Гистограмма {df[i].name}')
        axes[0].set_xlabel(df[i].name)
        axes[0].set_ylabel('Частота')

        df[i].plot(kind='box', ax=axes[1])
        axes[1].set_title(f'Boxplot {df[i].name}')
        
        plt.tight_layout()
        plt.show()

In [ ]:
def cat_bar(df, lst):
     for i in lst:
        plt.figure()
        df[i].value_counts().plot(kind='bar')
        plt.title(df[i].name)
        plt.xlabel(df[i].name)
        plt.ylabel('Частота')
        plt.show()

In [ ]:
market_file_num = market_file.select_dtypes(exclude='object').columns
market_file_cat = market_file.select_dtypes('object').columns
market_money_num = market_money.select_dtypes(exclude='object').columns
market_money_cat = market_money.select_dtypes('object').columns
market_time_num = market_time.select_dtypes(exclude='object').columns
market_time_cat = market_time.select_dtypes('object').columns
money_num = money.select_dtypes(exclude='object').columns
money_cat = money.select_dtypes('object').columns

In [ ]:
num_hist(market_file, market_file_num)

Нормальное распределение имеет. только признак 'ошибка_сервиса', выбросов нет

In [ ]:
cat_bar(market_file, market_file_cat)

В целевой переменной имеется небольшой дисбаланс классов

In [ ]:
num_hist(market_money, market_money_num)

Удалим выброс в выручке

In [ ]:
market_money = market_money.query('выручка < 80000')

In [ ]:
num_hist(market_money, market_money_num)

In [ ]:
cat_bar(market_money, market_money_cat)

Периоды распределены равномерно

In [ ]:
num_hist(market_time, market_time_num)

In [ ]:
cat_bar(market_time, market_time_cat)

Периоды распределены равномерно

In [ ]:
num_hist(money, money_num)

Прибыль имеет нормальное распределение. Аномалий нет

## Объединение таблиц

Просуммируем занчения в market_money и market_time для одинаковых id и присоединим к market_file по id

In [ ]:
df = market_file.\
merge(market_money.query('период == "препредыдущий_месяц"').pivot_table(index='id', 
                                                                      values='выручка', aggfunc='sum').reset_index().\
     rename(columns={'выручка': 'выручка_препред'}))
df = df.\
merge(market_money.query('период == "предыдущий_месяц"').pivot_table(index='id', 
                                                                      values='выручка', aggfunc='sum').reset_index().\
     rename(columns={'выручка': 'выручка_пред'}))
df = df.\
merge(market_money.query('период == "текущий_месяц"').pivot_table(index='id', 
                                                                      values='выручка', aggfunc='sum').reset_index().\
     rename(columns={'выручка': 'выручка_текущий'}))
df = df.\
merge(market_time.query('период == "предыдущий_месяц"').pivot_table(index='id', 
                                                                      values='минут', aggfunc='sum').reset_index().\
     rename(columns={'минут': 'минут_пред'}))
df = df.\
merge(market_time.query('период == "текущий_месяц"').pivot_table(index='id', 
                                                                      values='минут', aggfunc='sum').reset_index().\
     rename(columns={'минут': 'минут_текущий'}))

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>



Успех 👍:



 

- правильно что использован pivot_table,  наверное самый оптимальный метод для  задачи что то повернуть


 

In [ ]:
df.head()

Удалим из данных пользователей, которые не совершали покупки ни в одном из месяцев

In [ ]:
df = df.query('выручка_препред != 0 and выручка_текущий !=0 and выручка_пред != 0')

## Корреляционный анализ

In [ ]:
cor = df.drop('id', axis=1).corr(method='spearman')

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cor, annot=True, cmap='coolwarm');
plt.title('Кореляционная матрица')
plt.show()

Построим матрицы отдельно для пользователей со снижающейся и с прежней активностью

In [ ]:
cor_lower = df.drop('id', axis=1).query('покупательская_активность == "Снизилась"').corr(method='spearman')

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cor_lower, annot=True, cmap='coolwarm');
plt.title('Кореляционная матрица')
plt.show()

In [ ]:
cor_stable = df.drop('id', axis=1).query('покупательская_активность != "Снизилась"').corr(method='spearman')

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cor_stable, annot=True, cmap='coolwarm');
plt.title('Кореляционная матрица')
plt.show()

In [ ]:
phik_matrix = df.drop('id', axis=1).phik_matrix(interval_cols=df.drop('id', axis=1).select_dtypes(exclude='object').columns)

plot_correlation_matrix(phik_matrix.values, 
                        x_labels=phik_matrix.columns, 
                        y_labels=phik_matrix.index, 
                        vmin=-1, vmax=1, title="Phik матрица", figsize=(15, 10))

plt.show()

За исключением выручки и минут в разные периоды, значительной зависимости между признаками не наблюдается. Мультиколлинеарности нет

## Использование пайплайнов

Сначала разделим выборку

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['покупательская_активность', 'id'], axis=1),
    df['покупательская_активность'],
    test_size = 0.2, 
    random_state = 42,
    stratify = df['покупательская_активность'])

In [ ]:
for i in df.select_dtypes('object'):
    print(i, df[i].unique())

In [ ]:
ohe_columns = ['разрешить_сообщать', 'популярная_категория']
ord_columns = ['тип_сервиса']
num_columns = X_train.select_dtypes(exclude='object').columns

Создадим пайплайны

In [ ]:
ohe_pipe = Pipeline(
    [
     ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore'))
    ]
    )

ord_pipe = Pipeline(
    [
     ('ord',  OrdinalEncoder(
                categories=[
                    ['стандарт', 'премиум'],
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        )
    ]
)

In [ ]:
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('ord', ord_pipe, ord_columns),
     ('num', MinMaxScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LogisticRegression(random_state=42))
])

In [ ]:
param_distributions = [
    {
        'preprocessor__num': [MinMaxScaler(), StandardScaler()],
        'models': [LogisticRegression(random_state=42)],
        'models__C': [0.01, 0.1, 1, 10], 
        'models__penalty': ['l1', 'l2'],
        'models__solver': ['liblinear']
    },
    {
        'preprocessor__num': [MinMaxScaler(), StandardScaler()],
        'models': [DecisionTreeClassifier(random_state=42)],
        'models__max_depth': range(2, 12),
        'models__min_samples_split': range(2, 10)
    },
    {
        'preprocessor__num': [MinMaxScaler(), StandardScaler()],
        'models': [SVC()],
        'models__C': [0.01, 0.1, 1, 10],  
        'models__kernel': ['linear', 'rbf'],
    },
     {
        'preprocessor__num': [MinMaxScaler(), StandardScaler()],
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': [2, 3, 5, 10, 100, 1000],  
    }
]

In [ ]:
recall_scorer = make_scorer(recall_score, pos_label='Снизилась')

In [ ]:
random_search = RandomizedSearchCV(
    pipe_final, 
    param_distributions=param_distributions,
    n_iter=100, 
    cv=5, 
    scoring=recall_scorer, 
    n_jobs=-1, 
    random_state=35
)

In [ ]:
%%time
random_search.fit(X_train, y_train)

In [ ]:
random_search.classes_

In [ ]:
random_search.best_params_

In [ ]:
f'Метрика roc_auc у лучшей модели - {random_search.best_score_}'

Лучше всего себя показала модель логистической регрессии с l1 регуляризацией и параметром C, равным 0.01

In [ ]:
model = random_search.best_estimator_.named_steps['models']

In [ ]:
X_test_transformed = random_search.best_estimator_.named_steps['preprocessor'].transform(X_test)

In [ ]:
y_pred = model.predict(X_test_transformed)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred, pos_label='Снизилась')

Метрики хорошие

## Анализ важности признаков

Построим график SHAP

In [ ]:
feature_names = random_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
feature_names

In [ ]:
explainer = shap.LinearExplainer(model, X_test_transformed, feature_names=feature_names)
shap_values = explainer(X_test_transformed)
shap.plots.bar(shap_values, max_display=17)

Наибольшее влияние имеют признаки "страниц за визит", "средний_просмотр_категорий_за_визит" и "минут". Нулевое влияние имеют выручка за текущий месяц, "маркет_актив" за текущий месяц и "разрешить_сообщать", а также категориальные признаки

1. Можно сделать вывод, что маркетинговая активность в последний месяц перестала влиять на пользовательскую, поэтому имеет смысл пересмотреть стратегию
2. Больше всего на пользовательскую влияют факторы, задерживающие их на сайте. Можно улучшить рекомендательную систему, что поможет повышать активность

## Сегментация покупателей

Добавим новые столбцы, по которым будут составляться категории: вероятность снижения активности и прибыль

In [ ]:
model.classes_

In [ ]:
df_transformed = random_search.best_estimator_.named_steps['preprocessor'].\
transform(df.drop(['id', 'покупательская_активность'], axis=1))

In [ ]:
proba = model.predict_proba(df_transformed)

In [ ]:
df['вероятность_снижения_активности'] = proba[:, 1]

In [ ]:
df = df.merge(money)

In [ ]:
df.head()

In [ ]:
def num_hist_group(df1, df2, lst):
    for i in lst:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
        
        df1[i].plot(kind='hist', bins=15, ax=axes[0], alpha=0.7)
        axes[0].set_title(f'Гистограмма {df1[i].name} группы')
        axes[0].set_xlabel(df1[i].name)
        axes[0].set_ylabel('Частота')

        df2[i].plot(kind='hist', bins=15, ax=axes[1], alpha=0.7)
        axes[1].set_title(f'Гистограмма {df2[i].name}')
        axes[1].set_xlabel(df2[i].name)
        axes[1].set_ylabel('Частота')
        
        plt.tight_layout()
        plt.show()

In [ ]:


def cat_bar_group(df1, df2, lst):
    for i in lst:
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
        
        df1[i].value_counts().plot(kind='bar', ax=axes[0])
        axes[0].set_title(f'{df1[i].name} группы')
        axes[0].set_xlabel(df1[i].name)
        axes[0].set_ylabel('Частота')

        df2[i].value_counts().plot(kind='bar', ax=axes[1])
        axes[1].set_title(f'общий {df2[i].name}')
        axes[1].set_xlabel(df2[i].name)
        axes[1].set_ylabel('Частота')
        
        plt.tight_layout()
        plt.show()


Выделим сегменты
1. Группа клиентов с высокой долей покупок по акции и высокой вероятностью снижения покупательской активности.



In [ ]:
group1 = df.query('акционные_покупки > 0.85 and вероятность_снижения_активности > 0.7')

In [ ]:
group1.describe()

In [ ]:
df_num = df.select_dtypes(exclude='object')
df_cat = df.select_dtypes('object')

In [ ]:
num_hist_group(group1, df, df_num)

In [ ]:
cat_bar_group(group1, df, df_cat)

Можно заметить, что пользователи группы просматривают меньше страниц и товаров, когда пользуютя сайтом. Завлечь их можно с помощью увеличения интересного предложения, например, в категории "товары для детей", так как она является самой популярной у группы

2. Группа клиентов, которые покупают только технику, то есть товары с длинным жизненным циклом.

In [ ]:
group2 = df.query('популярная_категория == "Техника для красоты и здоровья"')

In [ ]:
group2.describe()

In [ ]:
num_hist_group(group2, df, df_num)

In [ ]:
cat_bar(group2, df, df_cat)

Эта группа заинтересована в том, чтобы продолжать активность. Однако не так много из пользователей этой группы имеют тип сервиса "премиум". Может быть полезным продвигать этот тип.

3. Группа клиентов с высокой вероятностью снижения покупательской активности и наиболее высокой прибыльностью. Проанализируем в чем отличие  между ними и клиентами с высокой заинтересованностью

In [ ]:
df['прибыль'].quantile(0.7)

In [ ]:
group3 = df.query('вероятность_снижения_активности > 0.7 and прибыль >= 4.3')

In [ ]:
group3_ct = df.query('вероятность_снижения_активности < 0.4 and прибыль >= 4.3')

In [ ]:
group3.describe()

In [ ]:
num_hist_group(group3, group3_ct, df_num)

In [ ]:
cat_bar_group(group3, group3_ct, df_cat)

Можно заметить, что больше всего прибыли приносят пользователи, покупающие товары для детей и косметику с аксесуарами. Пользователи этой группы также просматривают меньше товаров, посещая сайт. Вероятно, пользователи этой группы не смогли найти подходящие для них товары, либо цена их неустраивала. Следует расширить предложение в этих категориях, ведь пользователей, предпочитающих покупать домашний текстиль куда больше устраивают условия

## Общий вывод


Требовалось обучить модель, рассчитывающую вероятность снижения активности у пользователей.

Для этого в данные были добавлены новые признаки из смежных таблиц, показывающие прибыль и время пребывания на сайте в текущий и предыдущие месяцы.

Кореляционный анализ показал, что данные подходят для обучения модели.

Для того, чтобы найти подходящую модель, были перебраны модели логистической регрессии, k ближайшиз соседей, опорных векторов и дерево решений с разными параметрами регуляризации, другими гиперпараметрами и алгоритмами масштабирования данных.

Наиболее подходящей для задачи оказалась модель логистической регрессии с l1-регуляризацией и параметром C=0.01. 

После предсказаний вероятности снижения активности, пользователи были разделены по сегментам, связанных схожими значениями определенных характеристик. Был сделан вывод, что пользователей определенных групп можно привлечь более выгодными условиями продажи их любимых категорий товаров.